In [7]:
from pydub import AudioSegment
from scipy import signal
from sklearn.utils import shuffle
import re
import pandas as pd
import numpy as np
from glob import iglob
import statistics as stats
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import librosa
from librosa import display
from sklearn.metrics import confusion_matrix
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Embedding, Activation
from keras.layers import Conv1D, Conv2D, MaxPooling2D, ConvLSTM2D, LSTM, Bidirectional, GRU, TimeDistributed
from keras.optimizers import sgd, Adam
from keras.engine.input_layer import Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
import keras.backend as K
from keras import regularizers
from keras.layers import Lambda
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.core import Activation, Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
import tensorflow as tf
import itertools
import math
import time
import os
import json

In [8]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:XLA_GPU:0', '/device:XLA_CPU:0']


In [19]:
def segment_audio(ds_total, transcript_directory, audio_directory, audio_out):
    input_array = []
    person_id_list = []
    level_array = []
    
    filenames = os.listdir(transcript_directory)
    
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
        
    for filename in filenames:
        transcript_path = os.path.join(transcript_directory, filename)
        transcript = pd.read_csv(transcript_path, sep='\t')
        m = re.search("(\d{3})_TRANSCRIPT.csv", filename)
        if m:
            """
            person_id = int(m.group(1))            
            person_audio = AudioSegment.from_wav(audio_directory + str(person_id) + '_AUDIO.wav')
            lines = len(transcript)
            combined_sounds = AudioSegment.empty()
            for i in range(0, lines):
                row = transcript.iloc[i]
                if row["speaker"] == "Participant":
                    t1 = int(float(row["start_time"]) * 1000)
                    t2 = int(float(row["stop_time"]) * 1000)
                    newAudio = person_audio[t1:t2]
                    combined_sounds += newAudio
                    
            combined_sounds.export(audio_out + str(person_id) + '.wav', format="wav")
            """
            w = AudioSegment.from_wav(audio_out + str(person_id) + '.wav')
            input_array.append(w)
            person_id_list.append(person_id)
            level_array.append(ds_total.loc[ds_total["Participant_ID"] == person_id, "level"].item())
    return input_array, person_id_list, level_array

In [10]:
num_classes = 5
data_path = "/home/nevronas/IG-GPUshare/Amit/transcripts/"
audio_path = "/home/nevronas/IG-GPUshare/Amit/audio/"

In [11]:
def load_avec_dataset_file(path, score_column):
    ds = pd.read_csv(path, sep=',')
    ds['level'] = pd.cut(ds[score_column], bins=[-1,0,5,10,15,25], labels=[0,1,2,3,4])
    ds['PHQ8_Score'] = ds[score_column]
    ds['cat_level'] = keras.utils.to_categorical(ds['level'], num_classes).tolist()
    ds = ds[['Participant_ID', 'level', 'cat_level', 'PHQ8_Score']]
    ds = ds.astype({"Participant_ID": float, "level": int, 'PHQ8_Score': int})
    return ds

In [12]:
train = load_avec_dataset_file('/home/nevronas/IG-GPUshare/Amit/train_split_Depression_AVEC2017.csv', 'PHQ8_Score')
dev = load_avec_dataset_file('/home/nevronas/IG-GPUshare/Amit/dev_split_Depression_AVEC2017.csv', 'PHQ8_Score')
test = load_avec_dataset_file('/home/nevronas/IG-GPUshare/Amit/full_test_split.csv', 'PHQ_Score')
print("Size: train= {}, dev= {}, test= {}".format(len(train), len(dev), len(test)))
train.head()

Size: train= 107, dev= 35, test= 47


,Participant_ID,level,cat_level,PHQ8_Score
0,303.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
1,304.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",6
2,305.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",7
3,310.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",4
4,312.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",2


In [13]:
ds_total = pd.concat([train,dev,test])
total_phq8 = len(ds_total)
print("Total size = {}".format(total_phq8))

Total size = 189


In [14]:
ds_total.to_csv(data_path + 'ds_total.csv', sep='\t')
print("File was created")

File was created


In [18]:
input_array, person_id_list, level_array = segment_audio(ds_total, data_path , audio_path, "/home/nevronas/IG-GPUshare/Amit/audio_cut/")

In [20]:
window_size = 100 * 1000
windows_path = "/home/nevronas/IG-GPUshare/Amit/audio_windows/"
for audio_file, person_id in zip(input_array, person_id_list):
    no_of_samples = len(audio_file)/window_size
    start = 0
    end = window_size
    for sample_number in range(int(no_of_samples)):
        audio_window = AudioSegment.empty()
        audio_window = audio_file[start:end]
        start = start + window_size
        end = end + window_size
        audio_window.export(windows_path + str(person_id) +'_'+ str(sample_number) + '.wav', format="wav")
    
    if len(audio_file) < end and end-len(audio_file)<=90:
        audio_window = AudioSegment.empty()
        audio_window = audio_file[start:len(audio_file)]
        np.concatenate((audio_window, np.zeros(shape=(end - len(audio_file), 1))))
        audio_window.export(windows_path + str(person_id) +'_'+ str(sample_number) + '.wav', format="wav")
    
    print(person_id)
    

458
472
388
402
351
405
465
318
313
345
448
327
456
337
396
412
300
387
349
330
363
302
359
382
442
389
344
417
484
401
311
409
315
471
447
461
374
303
469
489
320
352
367
425
371
380
321
455
454
373
370
479
481
375
449
424
433
406
446
435
334
423
421
411
420
333
385
332
336
443
483
430
312
341
308
356
391
445
379
397
350
400
486
335
340
451
463
473
365
383
407
339
434
353
377
427
314
474
444
354
384
392
366
491
457
459
482
324
490
368
323
426
428
403
399
419
470
467
476
395
306
357
393
322
441
328
329
355
422
462
464
404
372
317
439
343
436
346
331
466
325
431
386
475
378
369
316
492
326
480
487
310
361
381
413
338
414
477
376
429
488
468
360
452
438
410
415
390
408
485
450
453
440
432
304
478
364
319
362
347
305
437
301
358
307
348
309
418
416


In [32]:
train_audio_files= {}
test_audio_files= {}
dev_audio_files= {}

windows_path = "/home/nevronas/IG-GPUshare/Amit/audio_windows/"

for person_id, level in zip(train['Participant_ID'], train['level']):
    audio_content = []
    for i, wav_filename in enumerate(iglob(os.path.join(windows_path, str(int(person_id)) + "**.wav"))):
        audio_content.append(wav_filename)
    print(int(person_id))
    train_audio_files[int(person_id)] = audio_content

303
304
305
310
312
313
315
316
317
318
319
320
321
322
324
325
326
327
328
330
333
336
338
339
340
341
343
344
345
347
348
350
351
352
353
355
356
357
358
360
362
363
364
366
368
369
370
371
372
374
375
376
379
380
383
385
386
391
392
393
397
400
401
402
409
412
414
415
416
419
423
425
426
427
428
429
430
433
434
437
441
443
444
445
446
447
448
449
454
455
456
457
459
463
464
468
471
473
474
475
478
479
485
486
487
488
491


In [33]:
for person_id, level in zip(dev['Participant_ID'], dev['level']):
    audio_content = []
    for i, wav_filename in enumerate(iglob(os.path.join(windows_path, str(int(person_id)) + "**.wav"))):
        audio_content.append(wav_filename)
    dev_audio_files[int(person_id)] = audio_content

In [34]:
for person_id, level in zip(test['Participant_ID'], test['level']):
    audio_content = []
    for i, wav_filename in enumerate(iglob(os.path.join(windows_path, str(int(person_id)) + "**.wav"))):
        audio_content.append(wav_filename)
    test_audio_files[int(person_id)] = audio_content

In [35]:
AUDIO_LENGTH = window_size
def m5(num_classes=5):
    print('Using Model M5')
    m = Sequential()
    m.add(Conv1D(128,
                 input_shape=[AUDIO_LENGTH, 1],
                 kernel_size=80,
                 strides=4,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(128,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(256,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(512,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Lambda(lambda x: K.mean(x, axis=1)))  # Same as GAP for 1D Conv Layer
    m.add(Dense(num_classes, activation='softmax'))
    return m

In [36]:
model = m5(num_classes=5)
model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

Using Model M5


In [49]:
def read_audio_from_filename(filename, target_sr = 1000):
    audio, _ = librosa.load(filename, sr=target_sr, mono=True)
    audio = audio.reshape(-1, 1)
    return audio

In [50]:

x_tr = []
y_tr = []


train_y = {}
for person_id, level in zip(train['Participant_ID'], train['level']):
    train_y[int(person_id)] = level
    

for item in train_audio_files.items():
    for file in item[1]:
        print(file)
        x_tr.append(read_audio_from_filename(file))
        y_tr.append(train_y[item[0]])

y_tr = to_categorical(y_tr, num_classes=num_classes)
       


# reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, min_lr=0.0001, verbose=1)
# batch_size = 128
# model.fit(x=x_tr,
#           y=y_tr,
#           batch_size=batch_size,
#           epochs=400,
#           verbose=1,
#           shuffle=True,
#           validation_data=(x_te, y_te),
#           callbacks=[reduce_lr])

/home/nevronas/IG-GPUshare/Amit/audio_windows/303_0.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/303_3.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/303_1.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/303_2.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/303_4.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/303_5.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/304_0.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/304_2.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/304_1.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/305_7.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/305_5.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/305_8.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/305_0.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/305_4.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/305_10.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/305_6.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/305_9.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/3

/home/nevronas/IG-GPUshare/Amit/audio_windows/366_0.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/366_4.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/366_1.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/366_6.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/366_5.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/368_7.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/368_2.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/368_1.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/368_6.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/368_0.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/368_3.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/368_4.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/368_8.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/368_5.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/369_5.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/369_4.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/369_2.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/36

/home/nevronas/IG-GPUshare/Amit/audio_windows/429_0.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/429_1.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/429_3.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/429_2.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/430_1.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/430_3.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/430_2.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/430_0.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/433_1.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/433_0.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/434_5.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/434_7.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/434_0.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/434_6.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/434_1.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/434_4.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/434_3.wav
/home/nevronas/IG-GPUshare/Amit/audio_windows/43

In [51]:
x_tr = np.array(x_tr)
y_tr = np.array(y_tr)
print('x_tr.shape =', x_tr.shape)
print('y_tr.shape =', y_tr.shape)

x_tr.shape = (415, 100000, 1)
y_tr.shape = (415, 5)


In [52]:
x_te = []
y_te = []
test_y = {}
for person_id, level in zip(test['Participant_ID'], test['level']):
    test_y[int(person_id)] = level
    

for item in test_audio_files.items():
    for file in item[1]:
        x_te.append(read_audio_from_filename(file))
        y_te.append(test_y[item[0]])

y_te = to_categorical(y_te, num_classes=num_classes)

In [53]:
x_te = np.array(x_te)
y_te = np.array(y_te)
print('x_te.shape =', x_te.shape)
print('y_te.shape =', y_te.shape)

x_te.shape = (218, 100000, 1)
y_te.shape = (218, 5)


In [1]:
# if the accuracy does not increase over 10 epochs, reduce the learning rate by half.
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, min_lr=0.0001, verbose=1)
batch_size = 128
model.fit(x=x_tr,
          y=y_tr,
          batch_size=batch_size,
          epochs=400,
          verbose=1,
          shuffle=True,
          validation_data=(x_te, y_te),
          callbacks=[reduce_lr])

NameError: name 'ReduceLROnPlateau' is not defined